In [ ]:
import os
import xarray as xr
import numpy as np
import salvus.namespace as sn
import salvus.mesh as sm
import generator
from salvus.toolbox.helpers.wavefield_output import WavefieldOutput, wavefield_output_to_xarray
from salvus.flow.simple_config.boundary import Absorbing
from salvus.mesh.unstructured_mesh_utils import extract_model_to_regular_grid

SALVUS_FLOW_SITE_NAME = os.environ.get("SITE_NAME", "YOUR_SITE")
PROJECT_DIR = "YOUR_PROJECT_DIR"
save_path = "../data/"  # where to save the generated data

In [ ]:
# Simulation setting
center_frequency = 3.0  # Ricker wavelet
max_frequency_in_hertz = 6.0
start_time_in_seconds = -0.5
end_time_in_seconds = 2.0
time_step_in_seconds = 2e-3
sampling_interval_in_time_steps = 25
elements_per_wavelength = 1.0
tensor_order = 4  # polynomial degree, must be one of 1, 2, 4
ranks_per_job = 2

# Absorbing boundary condition
reference_velocity = 1500
number_of_wavelengths = 3.5
abwidth = sm.absorbing_boundary.get_absorbing_boundary_width(velocity=reference_velocity, 
                                                             lowest_frequency=center_frequency, 
                                                             number_of_wavelengths=number_of_wavelengths)
extra = 250
abwidth = abwidth + extra

# Full domain = computational domain + boundaries
a = 5000
xrange = yrange = a + 2 * abwidth
zrange = a + abwidth
spacing = 80
nx = ny = round(xrange / spacing + 1)
nz = round(zrange / spacing + 1)

In [ ]:
for iS in range(30000):  # number of simulations 
    !rm -rf YOUR_PROJECT_DIR

    x = y = np.linspace(-abwidth, a+abwidth, nx)
    z = np.linspace(0, zrange, nz)  # depth

    vs = generator.generate_3DGRF_vK(nx, ny, nz)
    vp_vs_ratio = generator.generate_3DGRF_simple(nx, ny, nz)
    vs = generator.perturbation_vs(vs)
    vp_vs_ratio = generator.perturbation_vpvsr(vp_vs_ratio)
    vp = generator.get_vp(vs, vp_vs_ratio)
    rho = np.ones_like(vs) * 2.7

    # Convert to SI
    vp = vp * 1000
    vs = vs * 1000
    rho = rho * 1000

    # Xarray dataset
    ds = xr.Dataset(
        coords={"x": x, "y": y, "z": z},
        data_vars={
            "VP": (["x", "y", "z"], vp),
            "VS": (["x", "y", "z"], vs),
            "RHO": (["x", "y", "z"], rho),
        },
    )

    # Generate source location
    srcxyz = np.array([np.random.uniform(0, a+1e-5), 
                       np.random.uniform(0, a+1e-5), 
                       np.random.uniform(0, a+1e-5)])

    wavelet = sn.simple_config.stf.Ricker(center_frequency=center_frequency)

    src = sn.simple_config.source.cartesian.MomentTensorPoint3D(
        x=srcxyz[0], y=srcxyz[1], z=srcxyz[2], mxx=1.0, myy=1.0, mzz=1.0, myz=0, mxz=0, mxy=0
    )

    recs = sn.simple_config.receiver.cartesian.Point3D(
        x=srcxyz[0],
        y=srcxyz[1],
        z=srcxyz[2],
        station_code=f"{50:05d}",
        fields=["displacement"],
        )

    # Absorbing boundary condition
    abc = Absorbing(width_in_meters=abwidth-extra, 
                    side_sets=['x0','x1','y0','y1','z1'],
                    taper_amplitude=center_frequency,
                    side_sets_are_axis_aligned=True)

    # Creating mesh
    model = sn.model.volume.cartesian.GenericModel(name="volume", data=ds)
    p = sn.Project.from_volume_model(PROJECT_DIR, model, True)

    sc = sn.SimulationConfiguration(
        name="volumetric_model",
        tensor_order=tensor_order,
        elements_per_wavelength=elements_per_wavelength,
        max_frequency_in_hertz=max_frequency_in_hertz,
        model_configuration=sn.ModelConfiguration(
            background_model=None,
            volume_models="volume",
        ),
        event_configuration=sn.EventConfiguration(
            wavelet=wavelet,
            waveform_simulation_configuration=sn.WaveformSimulationConfiguration(
                start_time_in_seconds=start_time_in_seconds,
                end_time_in_seconds=end_time_in_seconds,
                time_step_in_seconds=time_step_in_seconds,
                spectral_element_order=tensor_order,
                boundary_conditions=abc,
                ),
        ),
    )
        
    p.add_to_project(sc, overwrite=True)
    p.add_to_project(sn.Event(event_name="event_0", sources=src, receivers=recs))

    mesh = p.simulations.get_mesh("volumetric_model")

    p.simulations.launch(
        ranks_per_job=ranks_per_job,
        site_name=SALVUS_FLOW_SITE_NAME,
        events=p.events.list(),
        simulation_configuration="volumetric_model",
        extra_output_configuration={
            "volume_data": {
                "sampling_interval_in_time_steps": sampling_interval_in_time_steps,
                "fields": ["displacement"],
                },
        },
    )

    p.simulations.query(block=True)

    vout_path = (
        p.simulations.get_simulation_output_directory(
            simulation_configuration="volumetric_model", event=p.events.list()[0]
        )
        / "volume_data_output.h5"
    )
    wavefield_output = WavefieldOutput.from_file(filename=vout_path, field='displacement', output_type='volume')

    # Extracting data to regular grid
    nx1 = ny1 = nz1 = 64
    x1 = y1 = z1 = np.linspace(0, a, nx1)
    xx1, yy1, zz1 = np.meshgrid(x1, y1, z1, indexing='ij')
    points = np.stack((xx1.flatten(), yy1.flatten(), zz1.flatten()), axis=1)
    u = wavefield_output_to_xarray(wo=wavefield_output, points=points).values  # (nt, nc, nx, ny, nz)
    u = u.reshape(u.shape[0], u.shape[1], nx1, ny1, nz1)
    u = u.transpose(2, 3, 4, 0, 1)  # (nx, ny, nz, nt, nc)

    # Starting from t = 0 or t = start_time_in_seconds
    #i0 = round(-start_time_in_seconds / (time_step_in_seconds * sampling_interval_in_time_steps))
    i0 = 0
    np.save(save_path+"u_S{}.npy".format(iS), u[:, :, :, i0:, :])
    stf = wavelet.get_stf(sampling_rate_in_hertz=wavefield_output.sampling_rate, start_time_in_seconds=start_time_in_seconds, end_time_in_seconds=end_time_in_seconds)[1]
    np.save(save_path+"stf_S0.npy", stf[i0:])

    ds_extract_3d = xr.Dataset(
        coords={
            "x": x1,
            "y": y1,
            "z": z1,
        }
    )
    ds_extract_3d = extract_model_to_regular_grid(
        mesh, ds_extract_3d, ["VP", "VS"]
    )
    vp = ds_extract_3d.VP.values
    vs = ds_extract_3d.VS.values
    np.save(save_path+"vp_S{}.npy".format(iS), vp)
    np.save(save_path+"vs_S{}.npy".format(iS), vs)
    np.save(save_path+"src_S{}.npy".format(iS), srcxyz)

    !rm -rf YOUR_PROJECT_DIR
